In [1]:
from __future__ import print_function
import torch
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import torch.utils.data
import torch.nn.init as init
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pyDOE
from scipy import stats
import sys
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn import preprocessing
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.linear_model import ElasticNet
import sklearn
from sklearn.preprocessing import PolynomialFeatures

print( 'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

torch: 1.3.0+cpu cuda: False


## Start Dimensionality Reduction Related Code

## Load & Re-Scale Data Sets

In [2]:
""" Give the path to the training input files as the function argument, the function loads and rescales the initial data set"""
def load_rescale_data_sets(path):
    train_data = pd.read_csv(path, index_col=0).iloc[:,:-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_data = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_2_200Samples.csv'), index_col=0).iloc[:,:-1]
    cols = test_data.columns
    scalar = MinMaxScaler().fit(train_data)
    train_data = pd.DataFrame(scalar.transform (train_data)) 
    test_data = pd.DataFrame(scalar.transform (test_data)) 
    train_data.columns, test_data.columns = cols, cols
    return train_data, test_data

## Variational Autoencoders

In [3]:
""" Class Variational Autoencoders, which implements the Variational Autoencoders 
Neural Networks for Dimensionality Reduction """

class VAE(nn.Module):

    def __init__(self, x_dim, z_dim):

        super(VAE, self).__init__()

        self.x_dim = x_dim
        self.z_dim = z_dim
        
        self.h1_dim = int (self.x_dim - (1/5) * (self.x_dim - self.z_dim))
        self.h2_dim = int (self.x_dim - (2/5) * (self.x_dim - self.z_dim))
        self.h3_dim = int (self.x_dim - (3/5) * (self.x_dim - self.z_dim))
        self.h4_dim = int (self.x_dim - (4/5) * (self.x_dim - self.z_dim))
        

        #encoder
        self.enc = nn.Sequential( nn.Linear(self.x_dim , self.h1_dim), nn.Tanh(),
                                nn.Linear(self.h1_dim , self.h2_dim), nn.Tanh(),
                                nn.Linear(self.h2_dim , self.h3_dim), nn.Tanh(),
                                nn.Linear(self.h3_dim , self.h4_dim), nn.Tanh())
        
        self.enc_mean = nn.Sequential( nn.Linear(self.h4_dim, self.z_dim) )
        self.enc_std = nn.Sequential( nn.Linear(self.h4_dim, self.z_dim))

        #decoder
        self.dec = nn.Sequential(nn.Linear(self.z_dim , self.h4_dim), nn.Tanh(),
                                nn.Linear(self.h4_dim , self.h3_dim), nn.Tanh(),
                                nn.Linear(self.h3_dim , self.h2_dim), nn.Tanh(),
                                nn.Linear(self.h2_dim , self.h1_dim), nn.Tanh())
        
        self.dec_mean = nn.Sequential( nn.Linear( self.h1_dim, self.x_dim ))
        self.dec_std = nn.Sequential( nn.Linear(self.h1_dim, self.x_dim))
        

    def encode (self, x ):
        #print (x.shape)
        enc = self.enc(x.float())
        enc_mean = self.enc_mean(enc)
        enc_std = self.enc_std(enc)
        return enc_mean , enc_std


    def decode (self, z):
        dec = self.dec(z)
        dec_mean = self.dec_mean(dec)
        dec_std = self.dec_std(dec)
        return dec_mean , dec_std

    def forward(self, x):
        kld_loss = 0
        nll_loss = 0
        #encoder
        enc_mean , enc_std = self.encode(x)
        #sampling and reparameterization
        z = self._reparameterized_sample(enc_mean, enc_std)
        #decoder
        dec_mean , dec_std = self.decode(z)
        kld_loss += self._kld_gauss(enc_mean, enc_std.mul(0.5).exp_())
        nll_loss += self._nll_gauss(dec_mean, dec_std, x)
        return kld_loss, nll_loss,(enc_mean , enc_std),(dec_mean , dec_std) , z


    def _reparameterized_sample(self, mean, logvar):
        """using std to sample"""
        std = logvar.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mean)


    def _kld_gauss(self, mu, logvar):
        """Using std to compute KLD"""
        return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    def _nll_gauss(self, mean, logvar , x):
        return torch.sum( 0.5 * np.log (2 * np.pi) + 0.5 * logvar + (x-mean)**2 / (2 *  torch.exp(logvar)) )


## Train and Test the VAEs

In [4]:
""" Functions to Implement the Training and Testing of AEs, Based on Methods in Autoencoder Class"""
def train(epoch, train_loader, train_data, batch_size, model):
    train_loss = 0
    epoch_loss = np.zeros(int(len (train_data) / batch_size ))
    epoch_div = np.zeros(int(len (train_data) / batch_size))
    clip, learning_rate, seed, print_every, save_every  = 10, 1e-3 , 100, 10, 10
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for batch_idx, (data) in enumerate(train_loader):
        
        data = Variable(data)
        #forward + backward + optimize
        optimizer.zero_grad()
        kld_loss, nll_loss, lat, recon, _ = model(data)
        epoch_loss [batch_idx] = nll_loss
        epoch_div [batch_idx] = kld_loss
        loss = kld_loss + nll_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

            

        train_loss += loss.data
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    return epoch_loss, epoch_div, model
    
def test(epoch, test_loader, test_data, model):
    """uses test data to evaluate 
    likelihood of the model"""
    mean_kld_loss, mean_nll_loss = 0, 0
    epoch_loss = np.zeros(len(test_data))
    epoch_div = np.zeros(len(test_data))
    for i, (data) in enumerate(test_loader):                                           
        
        data = Variable(data.reshape(1,-1))
        kld_loss, nll_loss, _, _, _ = model(data)
        epoch_div [i] = kld_loss
        epoch_loss [i] = nll_loss
        mean_kld_loss += kld_loss.data
        mean_nll_loss += nll_loss.data

    mean_kld_loss /= len(test_loader.dataset)
    mean_nll_loss /= len(test_loader.dataset)

    print('====> Test set loss: KLD Loss = {:.4f}, NLL Loss = {:.4f} '.format(
        mean_kld_loss, mean_nll_loss))
    return epoch_loss, epoch_div

## Perform Dimensionality Reduction

In [5]:
""" This is the method that implements the dimensionality reduction based on Autoencoders """
def perform_dimensionality_reduction (hyper_paras, path):
    seed = 100
    # Change the 0.3 to 0.6 and 0.9 for 60 % and 90 % dimensionality reduction
    z_dim = int(50-0.3 * 50) # 30 % dimensionality reduction
    n_epochs, batch_size = hyper_paras
    train_data, test_data = load_rescale_data_sets(path)
    x_dim = train_data.shape[1]
    torch.manual_seed(seed)
    
    h1_dim = int (x_dim - (1/5) * (x_dim - z_dim))
    h2_dim = int (x_dim - (2/5) * (x_dim - z_dim))
    h3_dim = int (x_dim - (3/5) * (x_dim - z_dim))
    h4_dim = int (x_dim - (4/5) * (x_dim - z_dim))
    
    print (x_dim, h1_dim, h2_dim, h3_dim,h4_dim,z_dim)
    
    train_loader = torch.utils.data.DataLoader ( dataset = train_data.values ,  batch_size = batch_size , shuffle= True)
    test_loader = torch.utils.data.DataLoader (  dataset = test_data.values , shuffle= True)
    train_error = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
    train_div = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
    test_error , test_div  = np.zeros([n_epochs , test_data.shape[0]]) , np.zeros([n_epochs , test_data.shape[0]]) 
    for epoch in range(1, n_epochs + 1):
        #training + testing
        if (epoch==1): ## only for the first time, take the default model, all the next times in iteration, keep improving it
            model = VAE(x_dim, z_dim)
            
        tr = train(epoch, train_loader, train_data, batch_size, model)
        train_error [epoch-1 , :] = tr [0]
        train_div [epoch-1 , :] = tr [1] 
        model = tr[2]
        te = test(epoch, test_loader, test_data, model)
        test_error [epoch-1 , :] = te [0]
        test_div [epoch-1 , :] = te [1]
            
    train_lat = [ model (Variable(torch.tensor(train_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(train_data)) ]
    test_lat = [ model (Variable(torch.tensor(test_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(test_data)) ]
    train_lat = pd.DataFrame(torch.cat(train_lat).cpu().detach().numpy())
    test_lat = pd.DataFrame(torch.cat(test_lat).cpu().detach().numpy())
    cols = []
    for i in range(train_lat.shape[1]):
        cols.append(str('Z'+str(i+1)))
    train_lat.columns = cols
    test_lat.columns = cols
    train_lat.to_csv('VAEs_Kriging_50D_30%_latent_training.csv') # Change the path here depending upon dimensionality and reduction
    test_lat.to_csv('VAEs_Kriging_50D_30%_latent_test.csv') # Same here
    return train_error, train_div, test_error, test_div

## End Dimensionality Reduction Related Code

## Load New Reduced Data Sets for all Test Cases

In [6]:
def load_f2(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_2_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f3(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_3_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f7(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_7_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f9(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_9_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f10(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets/test_10_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f13(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets/test_13_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f15(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets/test_15_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f16(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets/test_16_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f20(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_20_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f24(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets/test_24_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

## Start Kriging Surrogate Modelling Code

In [7]:
''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred

""" Normalized Mean Absolute Error % """
def rmae(true, pred):
    return np.mean((abs(true-pred) / abs(true)) * 100)

""" This method implements and evaluates the Kriging Surrogate Model with RMAE """
def surrogate_model(train_data,test_data,true):
    kri_model, kri_pred = kriging(train_data,test_data.iloc[:,:-1])
    return rmae(true,kri_pred)

""" Implements all the surrogate models, i.e., for all test function, and returns the median of RMAE errors,
This median is used as the primary metric for Hyper-Parameters Optimization """
def perform_surrogate_modeling(paths, path_latent_train,path_latent_test):
    train_2, test_2, true_2 = load_f2(paths[0],path_latent_train,path_latent_test)
    rmae_2 = surrogate_model(train_2, test_2, true_2)
    
    train_3, test_3, true_3 = load_f3(paths[1],path_latent_train,path_latent_test)
    rmae_3 = surrogate_model(train_3, test_3, true_3)
    
    train_7, test_7, true_7 = load_f7(paths[2],path_latent_train,path_latent_test)
    rmae_7 = surrogate_model(train_7, test_7, true_7)
    
    train_9, test_9, true_9 = load_f9(paths[3],path_latent_train,path_latent_test)
    rmae_9 = surrogate_model(train_9, test_9, true_9)
    
    train_10, test_10, true_10 = load_f10(paths[4],path_latent_train,path_latent_test)
    rmae_10 = surrogate_model(train_10, test_10, true_10)
    
    train_13, test_13, true_13 = load_f13(paths[5],path_latent_train,path_latent_test)
    rmae_13 = surrogate_model(train_13, test_13, true_13)
    
    train_15, test_15, true_15 = load_f15(paths[6],path_latent_train,path_latent_test)
    rmae_15 = surrogate_model(train_15, test_15, true_15)
    
    train_16, test_16, true_16 = load_f16(paths[7],path_latent_train,path_latent_test)
    rmae_16 = surrogate_model(train_16, test_16, true_16)
    
    train_20, test_20, true_20 = load_f20(paths[8],path_latent_train,path_latent_test)
    rmae_20 = surrogate_model(train_20, test_20, true_20)
    
    train_24, test_24, true_24 = load_f24(paths[9],path_latent_train,path_latent_test)
    rmae_24 = surrogate_model(train_24, test_24, true_24)
    
    accuracy = [rmae_2,rmae_3,rmae_7,rmae_9,rmae_10,rmae_13,rmae_15,rmae_16,rmae_20,rmae_24]
    return accuracy

""" This is the function used for Hyper_Parameters_Optimization for both dimensionality reduction and surrogate modelling """
def hyper_parameters_optimization(hyper_dim,path_latent_train,path_latent_test):
    print ('Start Dimensionality Reduction:::')
    _ , _,_,_ = perform_dimensionality_reduction (hyper_dim, paths[0])
    print ('End Dimensionality Reduction:::')
    accuracy = perform_surrogate_modeling (paths,path_latent_train,path_latent_test)
    return accuracy

## End Surrogate Modelling Code

## Set Paths

In [8]:
# Change the Paths here depending upon the dimensionality {50D=1000,100D=2000,200D=4000} 
path_2 = "Data Generation/50 D/Training_Data_Sets/train_2_1000Samples.csv"
path_3 = "Data Generation/50 D/Training_Data_Sets/train_3_1000Samples.csv"
path_7 = "Data Generation/50 D/Training_Data_Sets/train_7_1000Samples.csv"
path_9 = "Data Generation/50 D/Training_Data_Sets/train_9_1000Samples.csv"
path_10 = "Data Generation/50 D/Training_Data_Sets/train_10_1000Samples.csv"
path_13 = "Data Generation/50 D/Training_Data_Sets/train_13_1000Samples.csv"
path_15 = "Data Generation/50 D/Training_Data_Sets/train_15_1000Samples.csv"
path_16 = "Data Generation/50 D/Training_Data_Sets/train_16_1000Samples.csv"
path_20 = "Data Generation/50 D/Training_Data_Sets/train_201000Samples.csv"
path_24 = "Data Generation/50 D/Training_Data_Sets/train_241000Samples.csv"
# Change the Paths here depending upon the dimensionality {50D=1000,100D=2000,200D=4000} 
path_latent_train = "VAEs_Kriging_50D_30%_latent_training.csv"
path_latent_test = "VAEs_Kriging_50D_30%_latent_test.csv"
paths = [path_2,path_3,path_7,path_9,path_10,path_13,path_15,path_16,path_20,path_24]

## Run the Code

In [9]:
hyper_dim = [35,10]
accuracy = hyper_parameters_optimization(hyper_dim,path_latent_train,path_latent_test)
print ('The Median accuracy:::'+str(np.median(accuracy)))

Start Dimensionality Reduction:::
50 47 44 41 38 35
Train Epoch: 1 [0/1000 (0%)]	 KLD Loss: 12.757650 	 NLL Loss: 56.122307
Train Epoch: 1 [100/1000 (10%)]	 KLD Loss: 10.805971 	 NLL Loss: 51.470576
Train Epoch: 1 [200/1000 (20%)]	 KLD Loss: 7.907580 	 NLL Loss: 45.718915
Train Epoch: 1 [300/1000 (30%)]	 KLD Loss: 5.745353 	 NLL Loss: 36.321095
Train Epoch: 1 [400/1000 (40%)]	 KLD Loss: 3.307683 	 NLL Loss: 27.365556
Train Epoch: 1 [500/1000 (50%)]	 KLD Loss: 3.056167 	 NLL Loss: 15.966243
Train Epoch: 1 [600/1000 (60%)]	 KLD Loss: 2.516834 	 NLL Loss: 12.149077
Train Epoch: 1 [700/1000 (70%)]	 KLD Loss: 1.898360 	 NLL Loss: 12.160123
Train Epoch: 1 [800/1000 (80%)]	 KLD Loss: 1.379841 	 NLL Loss: 9.552052
Train Epoch: 1 [900/1000 (90%)]	 KLD Loss: 1.311139 	 NLL Loss: 10.500414
====> Epoch: 1 Average loss: 30.0130
====> Test set loss: KLD Loss = 1.1168, NLL Loss = 9.8278 
Train Epoch: 2 [0/1000 (0%)]	 KLD Loss: 1.117300 	 NLL Loss: 11.750836
Train Epoch: 2 [100/1000 (10%)]	 KLD Loss: 

Train Epoch: 20 [700/1000 (70%)]	 KLD Loss: 0.026802 	 NLL Loss: 8.761021
Train Epoch: 20 [800/1000 (80%)]	 KLD Loss: 0.037007 	 NLL Loss: 7.644003
Train Epoch: 20 [900/1000 (90%)]	 KLD Loss: 0.031056 	 NLL Loss: 7.891474
====> Epoch: 20 Average loss: 8.2432
====> Test set loss: KLD Loss = 0.0234, NLL Loss = 8.6708 
Train Epoch: 21 [0/1000 (0%)]	 KLD Loss: 0.031836 	 NLL Loss: 6.256417
Train Epoch: 21 [100/1000 (10%)]	 KLD Loss: 0.025710 	 NLL Loss: 8.613961
Train Epoch: 21 [200/1000 (20%)]	 KLD Loss: 0.033434 	 NLL Loss: 9.816117
Train Epoch: 21 [300/1000 (30%)]	 KLD Loss: 0.027135 	 NLL Loss: 8.762604
Train Epoch: 21 [400/1000 (40%)]	 KLD Loss: 0.041815 	 NLL Loss: 7.823468
Train Epoch: 21 [500/1000 (50%)]	 KLD Loss: 0.034872 	 NLL Loss: 8.757090
Train Epoch: 21 [600/1000 (60%)]	 KLD Loss: 0.022697 	 NLL Loss: 9.294384
Train Epoch: 21 [700/1000 (70%)]	 KLD Loss: 0.035943 	 NLL Loss: 6.491380
Train Epoch: 21 [800/1000 (80%)]	 KLD Loss: 0.049581 	 NLL Loss: 8.315454
Train Epoch: 21 [90

## A Note on the Possible Values of the Hyper_Parameters

In [ ]:
""" Hyper-Parameters for Dimensionality Reduction::::

    Variational Autoencoders takes 2 hyper parameters: number of epochs and batch size respectively, 
    both of which are discrete variables. Below are the possible candidates. 
    
    
    number of epochs, (Discrete Variable), Possible Values: {10,20,30,...,100} # increase of 10 every time
    batch size: (Discrete Variable): Possible Values: {5,10,20,25,50,100} # or any value that divides 1000 perfectly
"""
pd.DataFrame(np.array(accuracy), columns = ['Accuracy']).to_csv('Accuracy.csv')